In [2]:
!sudo pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-0aao2ng1
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-0aao2ng1
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp37-none-any.whl size=8312 sha256=68da2e046e96e4769c3cdfae7d0b2b766f544ca4dbae45ac90106f544d4e474d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3wbfz51e/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [3]:
!pip install Keras-Applications

In [4]:
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow import keras 
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from tensorflow.keras import optimizers
from keras import preprocessing 
from keras import applications 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras_vggface.vggface import VGGFace
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

In [5]:
!unzip /content/data_500_compressed.zip

Archive:  /content/data_500_compressed.zip
replace data_500/val/happiness/img_6614.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
data = "data_500"

In [7]:
train_dir = data+"/train/"
train_datagen = ImageDataGenerator(rescale=1.0/255., 
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest'
                                    )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=2000,
                                                    classes=['surprise',
                                                            "fear",
                                                            "disgust",
                                                            "anger",
                                                            "neutrality",
                                                            "sadness",
                                                            "happiness"],
                                                    class_mode="categorical",
                                                    color_mode="rgb",
                                                    target_size=(224, 224))


Found 3150 images belonging to 7 classes.


In [8]:
X_train = train_generator[0][0]
y_train = train_generator[0][1]

In [9]:
def load_model(model_name='vgg16', lr=1e-4, dense_1=1020, dense_2=200):
    model = VGGFace(include_top=False, model=model_name, weights='vggface',
           input_shape=(224, 224, 3), pooling=None, classes=7)
    
    model.trainable = False 
    model = tf.keras.models.Sequential([
                            model,
                            keras.layers.Flatten(),
                            keras.layers.Dense(dense_1, activation='relu'),
                            keras.layers.Dense(dense_2, activation='relu'),
                            keras.layers.Dense(7, activation='softmax')
                                ])
    
    opt = optimizers.Adam(learning_rate=lr)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

#vg_face_model = load_model(model_name='senet50')

In [10]:
vg_wrap_model = KerasClassifier(build_fn=load_model, model_name='vgg16', lr=1e-4, dense_1=1020, dense_2=200, batch_size=32)

In [11]:
vg_wrap_model.get_params()

{'batch_size': 32,
 'build_fn': <function __main__.load_model>,
 'dense_1': 1020,
 'dense_2': 200,
 'lr': 0.0001,
 'model_name': 'vgg16'}

In [ ]:
tf.keras.backend.clear_session()
#epochs = [1]

param_grid ={'dense_1': [700, 1000, 1020, 1200],
 'dense_2': [300, 500, 700, 900],
 'lr': [1e-4, 2e-4, 5e-4, 7e-4],
 'model_name': ['vgg16','resnet50', 'senet50'],
}

grid = GridSearchCV(estimator=vg_wrap_model, param_grid=param_grid, verbose=1)

grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


13/13 [==============================] - 2s 98ms/step - loss: 1.9715 - accuracy: 0.1550


In [ ]:
grid_result.best_params_

In [ ]:
grid_result.best_score_